# Exercise 01:  Minimal high-level UQ example

This notebook demonstrates high-level UQ with CUQIpy on a 1D Deconvolution test problem.

## Learning objectives of this notebook:
- Get acquainted with the CUQIpy components needed to specify a Bayesian inverse problem.
- Run a high-level UQ analysis of an inverse problem.

## Table of contents: 
* [1. UQ in five lines!](#UQ5)
* [2. Loading a test problem](#TestProblem)
* [3. Specifying and solving a Bayesian inverse problem ](#Bayesian)
* [4. Changing the prior ★](#ChangingPrior)

First we import any python packages we need, here simply NumPy to handle array computations:

In [ ]:
import numpy as np

Then we import the functionality we need from CUQIpy:

In [ ]:
from cuqi.distribution import Gaussian, Laplace_diff
from cuqi.problem import BayesianProblem
from cuqi.testproblem import Deconvolution1D

## 1. UQ in five lines!   <a class="anchor" id="UQ5"></a> 

In just five lines of CUQIpy code we can:
- load a forward model and data from a 1D deconvolution test problem,
- specify a likelihood,
- specify a prior,
- formulate a Bayesian inverse problem, and
- run a UQ analysis:

In [ ]:
model, data, probInfo = Deconvolution1D.get_components(dim=50, phantom="Square")
likelihood = Gaussian(mean=model, std=0.05).to_likelihood(data)
prior = Gaussian(mean=np.zeros(50), std=0.2)
IP = BayesianProblem(likelihood, prior)
IP.UQ(exact=probInfo.exactSolution)

In the following sections of this notebook we break it down and take a slightly closer look at each step. The subsequent notebooks provide additional details.

## 2. Loading a test problem  <a class="anchor" id="TestProblem"></a> 

We specify a small 1D deconvolution test problem:

In [ ]:
model, data, probInfo = Deconvolution1D.get_components(dim=50, phantom="Square")

Note that the test problem can be configured (e.g. other phantoms, noise types and level etc.) by means of the inputs, see `help(Deconvolution1D)` for details.

Also note that a total of 6 test problems are currently available:
- `Deconvolution1D`: 1D periodic deconvolution problem.
- `Deconv_1D`:  1D deconvolution problem.
- `Heat_1D`:  Heat equation PDE problem.
- `Poisson_1D`: Poisson equation PDE problem.
- `Abel_1D`:  Abel equation PDE (1D rotationally symmetric CT) problem. 
- `ParBeamCT_2D`: 2D Computed Tomography problem using the ASTRA Tomography Toolbox

These are imported as follows:

In [ ]:
from cuqi.testproblem import Deconv_1D, Heat_1D, Poisson_1D, Abel_1D
from cuqi.astra import ParBeamCT_2D

and their calling signature is the same as for the `Deconvolution1D` problem, e.g. 
```
model, data, probInfo = Heat_1D.get_components()
```
Input arguments vary and default values are provided if left empty. Calling help of each testproblem, e.g., `help(Heat_1D)` will describe the test problem and the inputs it accepts.

For now, proceeding with the `Deconvolution1D` test problem, we take a look at the model and see that it is a CUQIpy LinearModel:

In [ ]:
model

We take a look at the data:

In [ ]:
data

The data is a `CUQIarray`, which is a normal NumPy array further equipped with a few utilities, such as Geometry, which allows us to do plotting conveniently:

In [ ]:
data.plot()

The last thing returned by the test problem was `probInfo` which contains additional information about the test problem, typically it includes the exact solution (phantom) and the exact data. We take a look at both:

In [ ]:
probInfo

In [ ]:
probInfo.exactSolution.plot()

In [ ]:
probInfo.exactData.plot()

Since `CUQIarray` is a NumPy array (technically subclassed from NumPy ndarray), we can do all computations that NumPy admits and still get a `CUQIarray`, for example take the difference between the data and exact data and call the plot method:

In [ ]:
(data-probInfo.exactData).plot()

## 3. Specifying and solving a Bayesian inverse problem  <a class="anchor" id="Bayesian"></a> 

The deconvolution test problem is a linear problem with additive noise:

$$ \mathbf{b} = \mathbf{A}\mathbf{x}+\mathbf{e},$$
where $\mathbf{A}\in\mathbb{R}^{n\times n}$, $\mathbf{x}, \mathbf{b}\in\mathbb{R}^n$ 
and 
$$
\mathbf{x}\sim\mathcal{N}(\mathbf{0},\sigma_x^2\mathbf{I}_n), \\
\mathbf{e}\sim\mathcal{N}(\mathbf{0},\sigma_e^2\mathbf{I}_n).$$

We need two components to specify a Bayesian problem: The prior and the likelihood.

The prior we specify as a simple IID Gaussian, where the dimension is set to match that of the domain of the model:

In [ ]:
std_prior = 0.2
prior = Gaussian(mean=np.zeros(model.domain_dim), std=std_prior)

The noise is also Gaussian. We can define the likelihood function by creating the Gaussian noise distribution with the model as mean and converting to a likelihood given the observed data:

In [ ]:
std_noise = 0.05
likelihood = Gaussian(mean=model, std=std_noise).to_likelihood(data)

The prior, likelihood are combined in a Bayesian inverse problem:

In [ ]:
IP = BayesianProblem(likelihood, prior)

The "completely non-expert approach" to solving (more detailed approaches described in later notebooks) is to simply run the UQ method:

In [ ]:
IP.UQ()

The `UQ` method looks at the components of the inverse problem, chooses a suitable sampler, samples the posterior and presents results visually.

To compare with the exact solution (if available) one can pass it as an input:

In [ ]:
IP.UQ(exact=probInfo.exactSolution)

## 4. Changing the prior ★ <a class="anchor" id="ChangingPrior"></a> 

It is straightforward to change components of the BayesianProblem. For example if we want to experiment with a different prior we can easily swap it out.

We specify a `Laplace_diff` prior, which is a Laplace distribution on differences between neighbouring function values:

In [ ]:
prior_lap = Laplace_diff(location=np.zeros(model.domain_dim), scale=0.01, bc_type='zero')

We update the prior of the inverse problem:

In [ ]:
IP.prior = prior_lap

And rerun the `UQ` method:

In [ ]:
IP.UQ(exact=probInfo.exactSolution)

Note how a different sampler was chosen due to the change of prior, and how the prior has changed the solution to be more similar to the exact solution.

#### Try yourself (optional):  
- Change phantom to another of the options provided by the TestProblem. Hint: use `help(Deconvolution1D)` to see which phantoms are available.
- Play with the parameters of the Gaussian and Laplace priors to see if a better solution can be obtained.
- Try other test problems of your choice as described in Section 2 of the notebook.